In [172]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Import Dataset

In [223]:
xls_path = pd.ExcelFile('/Users/dileepsathyan/Documents/GitHub/Assignments/Vroom Virtusa/Analytics_Dataset.xlsx')
df_car_sales   = pd.read_excel(xls_path, 'car_sales')
df_car_details = pd.read_excel(xls_path, 'car_details')
df_car_claims  = pd.read_excel(xls_path, 'car_claims')

In [177]:
df_car_sales.head()

,car_id_sales,country,merchant_id,selling_date,selling_week,payment_date,sell_price,transport_to_merchant,transport_date
0,3421775349,ES,NaN,2021-04-29,2021-17,2021-05-14,1098,1,2021-05-31
1,5316743932,DE,NaN,2021-08-06,2021-31,2021-08-17,435,1,2021-08-31
2,5785159431,FR,NaN,2021-06-03,2021-22,2021-06-08,11028,0,NaN
3,7008763171,DE,NaN,2021-06-07,2021-23,2021-06-08,21100,1,NaN
4,1417558732,DE,NaN,2021-08-31,2021-35,2021-09-02,1943,1,2021-09-09


In [178]:
df_car_details.head()

,car_id_details,has_tuning,has_airbags,has_alarm_system,fuel_type,gear_type,ac_type,xenon_light,navigation_system,radio_system
0,5096178328,1,1,0,1039,1141,1049,0,1100,3534.0
1,1879501444,0,1,1,1039,1138,1049,0,0,3534.0
2,8375508078,1,1,1,1039,1139,0,0,0,1606.0
3,1266330246,0,1,0,1040,1139,1050,0,0,3534.0
4,9079520882,0,0,0,1039,1141,0,0,1100,3534.0


In [179]:
df_car_claims.head()

,car_id_claims,claim_number,full_refunds,partial_refunds,voucher_amount,claim_status
0,7525691980,NaN,0,0,NaN,NaN
1,8543168465,C1AG79687,0,1,620.140528,CLOSED_FULLY_PROCESSED
2,7308726237,NaN,0,0,NaN,NaN
3,9070724843,NaN,0,0,NaN,NaN
4,6274074693,NaN,0,0,NaN,NaN


# Data Cleaning

## 1. Car_Sales Dataframe Cleaning

In [224]:
df_car_sales.shape

(70512, 9)

In [225]:
df_car_sales.drop_duplicates(inplace=True)
df_car_sales.shape

(69618, 9)

#### Convert to right datatypes

In [226]:
# df_car_sales['car_id_sales'] = df_car_sales['car_id_sales'].astype('str')
# df_car_sales['merchant_id'] = df_car_sales['merchant_id'].astype('str')

df_car_sales['selling_date'] = pd.to_datetime(df_car_sales['selling_date'])
df_car_sales['payment_date'] = pd.to_datetime(df_car_sales['payment_date'])
df_car_sales['transport_date'] = pd.to_datetime(df_car_sales['transport_date'])

# Rename primary column
df_car_sales.rename(columns={'car_id_sales': 'car_id'}, inplace=True)

df_car_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69618 entries, 0 to 70511
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   car_id                 69618 non-null  int64         
 1   country                69618 non-null  object        
 2   merchant_id            19088 non-null  float64       
 3   selling_date           69618 non-null  datetime64[ns]
 4   selling_week           69618 non-null  object        
 5   payment_date           69618 non-null  datetime64[ns]
 6   sell_price             69618 non-null  int64         
 7   transport_to_merchant  69618 non-null  int64         
 8   transport_date         32689 non-null  datetime64[ns]
dtypes: datetime64[ns](3), float64(1), int64(3), object(2)
memory usage: 5.3+ MB


In [227]:
# Notice that the same car_ids are repeated for different merchant_ids

df_car_sales[df_car_sales['car_id'] == 5805619905]

,car_id,country,merchant_id,selling_date,selling_week,payment_date,sell_price,transport_to_merchant,transport_date
24388,5805619905,BE,16191229.0,2021-07-31,2021-30,2021-08-03,22596,1,2021-08-04
46739,5805619905,BE,14307587.0,2021-07-31,2021-30,2021-08-03,22596,1,2021-08-04


In [228]:
# There are merchant_ids who bought so many cars.

df_car_sales.merchant_id.value_counts()

8819318.0     823
5482442.0      97
2593085.0      92
6717821.0      64
2491964.0      62
             ... 
3451474.0       1
6902952.0       1
5854417.0       1
6903110.0       1
10485917.0      1
Name: merchant_id, Length: 9102, dtype: int64

In [229]:
#### As long as we have no clarity on the correct Merchant who bought a car, it doesnt matter which record we delete.
#### Lets keep car_id only once and delete all following records for the sale.

df_car_sales = df_car_sales.drop_duplicates(subset='car_id', keep="first")

# Reset index
df_car_sales.reset_index(drop=True, inplace=True)

df_car_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69551 entries, 0 to 69550
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   car_id                 69551 non-null  int64         
 1   country                69551 non-null  object        
 2   merchant_id            19021 non-null  float64       
 3   selling_date           69551 non-null  datetime64[ns]
 4   selling_week           69551 non-null  object        
 5   payment_date           69551 non-null  datetime64[ns]
 6   sell_price             69551 non-null  int64         
 7   transport_to_merchant  69551 non-null  int64         
 8   transport_date         32654 non-null  datetime64[ns]
dtypes: datetime64[ns](3), float64(1), int64(3), object(2)
memory usage: 4.8+ MB


In [230]:
df_car_sales.head()

,car_id,country,merchant_id,selling_date,selling_week,payment_date,sell_price,transport_to_merchant,transport_date
0,3421775349,ES,NaN,2021-04-29,2021-17,2021-05-14,1098,1,2021-05-31
1,5316743932,DE,NaN,2021-08-06,2021-31,2021-08-17,435,1,2021-08-31
2,5785159431,FR,NaN,2021-06-03,2021-22,2021-06-08,11028,0,NaT
3,7008763171,DE,NaN,2021-06-07,2021-23,2021-06-08,21100,1,NaT
4,1417558732,DE,NaN,2021-08-31,2021-35,2021-09-02,1943,1,2021-09-09


## Car_Details Dataframe Cleaning

In [231]:
df_car_details.shape

(70512, 10)

In [232]:
df_car_details.drop_duplicates(inplace=True)
df_car_details.shape

(70034, 10)

In [233]:
df_car_details.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70034 entries, 0 to 70511
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   car_id_details     70034 non-null  int64  
 1   has_tuning         70034 non-null  int64  
 2   has_airbags        70034 non-null  int64  
 3   has_alarm_system   70034 non-null  int64  
 4   fuel_type          70034 non-null  int64  
 5   gear_type          70034 non-null  int64  
 6   ac_type            70034 non-null  int64  
 7   xenon_light        70034 non-null  int64  
 8   navigation_system  70034 non-null  int64  
 9   radio_system       43494 non-null  float64
dtypes: float64(1), int64(9)
memory usage: 5.9 MB


#### Convert to right datatypes

In [236]:
# df_car_details['car_id_details'] = df_car_details['car_id_details'].astype('str')

# Rename primary column
df_car_details.rename(columns={'car_id_details': 'car_id'}, inplace=True)
df_car_details.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70034 entries, 0 to 70511
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   car_id             70034 non-null  int64  
 1   has_tuning         70034 non-null  int64  
 2   has_airbags        70034 non-null  int64  
 3   has_alarm_system   70034 non-null  int64  
 4   fuel_type          70034 non-null  int64  
 5   gear_type          70034 non-null  int64  
 6   ac_type            70034 non-null  int64  
 7   xenon_light        70034 non-null  int64  
 8   navigation_system  70034 non-null  int64  
 9   radio_system       43494 non-null  float64
dtypes: float64(1), int64(9)
memory usage: 5.9 MB


In [255]:
## Check for duplicate records of same car_id

df_car_details[df_car_details.groupby('car_id')['car_id'].transform('size') > 1].sort_values(by='car_id').head(20)

,car_id,has_tuning,has_airbags,has_alarm_system,fuel_type,gear_type,ac_type,xenon_light,navigation_system,radio_system
49297,1021854212,0,0,0,1039,1141,0,0,1100,3534.0
25281,1021854212,0,0,0,1040,1141,0,0,1100,3534.0
19591,1071192762,1,1,1,1040,1139,1050,1089,1100,3534.0
37616,1071192762,1,1,1,1039,1139,1050,1089,1100,3534.0
20260,1092285954,1,1,0,1039,1138,1050,0,1100,NaN
62249,1092285954,1,1,0,1039,1138,1050,0,0,NaN
23369,1116277714,1,1,0,1039,1138,0,0,0,3534.0
62436,1116277714,1,1,0,1040,1138,0,0,1100,3534.0
50603,1130791238,1,1,0,1039,1141,1050,0,1100,1100.0
12934,1130791238,1,1,0,1040,1141,1050,0,1100,1100.0


In [316]:
# Subset the records with duplicate car_ids

df_cd_dup = df_car_details[df_car_details.groupby('car_id')['car_id'].transform('size') > 1].sort_values(by='car_id')
df_cd_dup.sort_values(by='car_id').head(10)

,car_id,has_tuning,has_airbags,has_alarm_system,fuel_type,gear_type,ac_type,xenon_light,navigation_system,radio_system
49297,1021854212,0,0,0,1039,1141,0,0,1100,3534.0
25281,1021854212,0,0,0,1040,1141,0,0,1100,3534.0
19591,1071192762,1,1,1,1040,1139,1050,1089,1100,3534.0
37616,1071192762,1,1,1,1039,1139,1050,1089,1100,3534.0
20260,1092285954,1,1,0,1039,1138,1050,0,1100,NaN
62249,1092285954,1,1,0,1039,1138,1050,0,0,NaN
23369,1116277714,1,1,0,1039,1138,0,0,0,3534.0
62436,1116277714,1,1,0,1040,1138,0,0,1100,3534.0
50603,1130791238,1,1,0,1039,1141,1050,0,1100,1100.0
12934,1130791238,1,1,0,1040,1141,1050,0,1100,1100.0


In [317]:
df_cd_dup.shape

(961, 10)

In [318]:
# Remove the duplicated car_ids from original dataframe to make it a deduplicated df.

df_cd_deduped = df_car_details.drop(df_cd_dup.index)
df_cd_deduped.shape

(69073, 10)

##### Notice from the above subset that fule_type and navigation_system are the 2 fields which cause duplication.

###### 1. Those who differ only by fuel_type, lets keep the first record of each car_id and delete the rest.

In [319]:
df_cd_dup = df_cd_dup.drop_duplicates(['car_id', 'navigation_system']).sort_values(by='car_id')

df_cd_dup.head(10)

,car_id,has_tuning,has_airbags,has_alarm_system,fuel_type,gear_type,ac_type,xenon_light,navigation_system,radio_system
49297,1021854212,0,0,0,1039,1141,0,0,1100,3534.0
19591,1071192762,1,1,1,1040,1139,1050,1089,1100,3534.0
20260,1092285954,1,1,0,1039,1138,1050,0,1100,NaN
62249,1092285954,1,1,0,1039,1138,1050,0,0,NaN
23369,1116277714,1,1,0,1039,1138,0,0,0,3534.0
62436,1116277714,1,1,0,1040,1138,0,0,1100,3534.0
50603,1130791238,1,1,0,1039,1141,1050,0,1100,1100.0
11699,1131021388,1,1,0,1039,1138,1050,0,0,3534.0
24110,1134904596,0,0,0,1040,1141,0,0,0,3534.0
48854,1175003748,1,1,0,1040,1138,1050,0,1100,1100.0


In [320]:
df_cd_dup.shape

(792, 10)

In [321]:
# Filter the uniques and append to the deduped dataframe & drop the uniques from the df_cd_dup

df_temp = df_cd_dup[df_cd_dup.groupby('car_id')['car_id'].transform('size') == 1]

df_cd_deduped = df_cd_deduped.append(df_temp)
df_cd_dup = df_cd_dup.drop(df_temp.index)
df_cd_dup.head(10)

,car_id,has_tuning,has_airbags,has_alarm_system,fuel_type,gear_type,ac_type,xenon_light,navigation_system,radio_system
20260,1092285954,1,1,0,1039,1138,1050,0,1100,NaN
62249,1092285954,1,1,0,1039,1138,1050,0,0,NaN
23369,1116277714,1,1,0,1039,1138,0,0,0,3534.0
62436,1116277714,1,1,0,1040,1138,0,0,1100,3534.0
54450,1189899240,0,0,0,1039,1141,0,0,0,3534.0
11135,1189899240,0,0,0,1040,1141,0,0,1100,3534.0
52138,1251496465,1,1,1,1039,1139,1050,1089,0,3534.0
67790,1251496465,1,1,1,1039,1139,1050,1089,1100,3534.0
8106,1257596002,0,0,0,1039,1141,0,0,0,3534.0
67243,1257596002,0,0,0,1039,1141,0,0,1100,3534.0


In [322]:
df_cd_dup.shape

(628, 10)

##### 2. Those who differ only by navigation_system, lets keep the first record of each car_id and delete the rest.

In [323]:
df_cd_dup = df_cd_dup.drop_duplicates(['car_id', 'fuel_type']).sort_values(by='car_id')

df_cd_dup.head(10)

,car_id,has_tuning,has_airbags,has_alarm_system,fuel_type,gear_type,ac_type,xenon_light,navigation_system,radio_system
20260,1092285954,1,1,0,1039,1138,1050,0,1100,NaN
23369,1116277714,1,1,0,1039,1138,0,0,0,3534.0
62436,1116277714,1,1,0,1040,1138,0,0,1100,3534.0
54450,1189899240,0,0,0,1039,1141,0,0,0,3534.0
11135,1189899240,0,0,0,1040,1141,0,0,1100,3534.0
52138,1251496465,1,1,1,1039,1139,1050,1089,0,3534.0
8106,1257596002,0,0,0,1039,1141,0,0,0,3534.0
69446,1257782013,1,1,0,1039,1138,0,0,0,NaN
29230,1257782013,1,1,0,1040,1138,0,0,1100,NaN
54861,1266182032,0,0,0,1039,1141,0,0,1100,3534.0


In [324]:
# Filter the uniques and append to the deduped dataframe & drop the uniques from the df_cd_dup

df_temp = df_cd_dup[df_cd_dup.groupby('car_id')['car_id'].transform('size') == 1]

df_cd_deduped = df_cd_deduped.append(df_temp)
df_cd_dup = df_cd_dup.drop(df_temp.index)
df_cd_dup.head(10)

,car_id,has_tuning,has_airbags,has_alarm_system,fuel_type,gear_type,ac_type,xenon_light,navigation_system,radio_system
23369,1116277714,1,1,0,1039,1138,0,0,0,3534.0
62436,1116277714,1,1,0,1040,1138,0,0,1100,3534.0
54450,1189899240,0,0,0,1039,1141,0,0,0,3534.0
11135,1189899240,0,0,0,1040,1141,0,0,1100,3534.0
69446,1257782013,1,1,0,1039,1138,0,0,0,NaN
29230,1257782013,1,1,0,1040,1138,0,0,1100,NaN
64274,1550589125,0,0,0,1039,1141,0,0,0,3534.0
2198,1550589125,0,0,0,1040,1141,0,0,1100,3534.0
47017,1675413187,0,0,0,1039,1141,0,0,0,3534.0
47934,1675413187,0,0,0,1040,1141,0,0,1100,3534.0


In [325]:
df_cd_dup.shape

(134, 10)

###### 3. Those who differ in both navigation_system & fuel_type, lets keep the record with 1100 as nav_system and delete the 0 values.

In [326]:
df_cd_dup = df_cd_dup[df_cd_dup['navigation_system'] != 0]
df_cd_deduped = df_cd_deduped.append(df_cd_dup)
df_cd_deduped.shape

(69551, 10)

In [338]:
df_car_details = df_cd_deduped

# Reset index
df_car_details.reset_index(drop=True, inplace=True)

df_car_details.shape

(69551, 10)

## Car_Claims Dataframe Cleaning

In [335]:
df_car_claims.shape

(70512, 6)

In [336]:
df_car_claims.drop_duplicates(inplace=True)
df_car_claims.shape

(70504, 6)

In [337]:
df_car_claims.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70504 entries, 0 to 70511
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   car_id_claims    70504 non-null  int64  
 1   claim_number     19974 non-null  object 
 2   full_refunds     70504 non-null  int64  
 3   partial_refunds  70504 non-null  int64  
 4   voucher_amount   15173 non-null  float64
 5   claim_status     19974 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 3.8+ MB


#### Convert to right datatypes

In [339]:
# df_car_claims['car_id_claims'] = df_car_claims['car_id_claims'].astype('str')
# df_car_claims['claim_number'] = df_car_claims['claim_number'].astype('str')

# Rename primary column
df_car_claims.rename(columns={'car_id_claims': 'car_id'}, inplace=True)

df_car_claims.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70504 entries, 0 to 70511
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   car_id           70504 non-null  int64  
 1   claim_number     19974 non-null  object 
 2   full_refunds     70504 non-null  int64  
 3   partial_refunds  70504 non-null  int64  
 4   voucher_amount   15173 non-null  float64
 5   claim_status     19974 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 3.8+ MB


In [342]:
df_car_claims['claim_status'].value_counts()

CLOSED_FULLY_PROCESSED                                                    7910
REJECTED                                                                  5026
CLOSED_NO_COMPENSATION                                                    3410
CREATED                                                                   1262
CLOSED_ITEMS_FOUND_AND_SENT                                                783
CLOSED_WAITING_FOR_MORE_PROOF                                              348
PROOF_NOT_RECEIVED                                                         309
WAITING_FOR_SALES_TO_NEGOTIATE_WITH_THE_MERCHANT                           234
WAITING_FOR_LOCAL_OPS_MD_DECISION                                          159
MISSING_ITEMS_IN_PROGRESS                                                  134
2ND_DECISION_WAITING_FOR_SALES_TO_NEGOTIATE_WITH_MERCHANT                   77
NEW                                                                         72
WAITING_FOR_CLAIMS_TEAM_2_EVALUATION                

In [ ]:
'NEW'
'CREATED'
'NEW_MISSING_ITEM_CLAIM'

'PROOF_NOT_RECEIVED'
'CLOSED_WAITING_FOR_MORE_PROOF'

'MORE_PROOF_RECEIVED'

'WAITING_FOR_CLAIMS_TEAM_1_EVALUATION'
'2ND_EVALUATION_CLAIMS_TEAM_2'
'WAITING_FOR_CLAIMS_TEAM_2_EVALUATION'

'WAITING_FOR_CARRIER_FEEDBACK'
'CLOSED_CARRIER_WILL_COMPENSATE'

'WAITING_FOR_LEGAL_TEAM'

'WAITING_FOR_SALES_TO_NEGOTIATE_WITH_THE_MERCHANT'
'2ND_DECISION_WAITING_FOR_SALES_TO_NEGOTIATE_WITH_MERCHANT'

'WAITING_FOR_LOCAL_OPS_MD_DECISION'
'2ND_DECISION_WAITING_FOR_LOCAL_OPS_MD_DECISION'

'MISSING_ITEMS_IN_PROGRESS'
'CLOSED_ITEMS_FOUND_AND_SENT'

'CLOSED_WAITING_FOR_MERCHANT_BANK_DETAILS'
'CLOSED_WAITING_FOR_CAR_DOCUMENTS_AND_BANK_DETAILS'
'WAITING_FOR_BANK_DETAILS_VALIDATION'
'CLOSED_WAITING_FOR_DOCUMENTS'

'CLOSED_FULLY_PROCESSED'
'CLOSED_WAITING_FOR_MONEY_TO_BE_SHIPPED_AND_COMPENSATION_TO_BE_RECORDED'

'CLOSED_OVER_TIME_LIMIT'
'CLOSED_WAITING_FOR_COMPENSATION_TO_BE_RECORDED'

'REJECTED'
'CLOSED_NO_COMPENSATION'

In [158]:
# Reset index
df_car_claims.reset_index(drop=True, inplace=True)

df_car_claims.head()

,car_id,claim_number,full_refunds,partial_refunds,voucher_amount,claim_status
0,7525691980,nan,0,0,NaN,NaN
1,8543168465,C1AG79687,0,1,620.140528,CLOSED_FULLY_PROCESSED
2,7308726237,nan,0,0,NaN,NaN
3,9070724843,nan,0,0,NaN,NaN
4,6274074693,nan,0,0,NaN,NaN


# Combine the dataframes to 1

In [159]:
df_car_sales.shape

(69618, 9)

In [160]:
df_car_details.shape

(70034, 10)

In [161]:
df_car_claims.shape

(70504, 6)

In [167]:
df_merged = pd.merge(df_car_claims, df_car_details, how='left', on='car_id')
# df_merged = pd.merge(df_merged, df_car_sales, how='inner', on='car_id')
df_merged.head()

,car_id,claim_number,full_refunds,partial_refunds,voucher_amount,claim_status,has_tuning,has_airbags,has_alarm_system,fuel_type,gear_type,ac_type,xenon_light,navigation_system,radio_system
0,7525691980,nan,0,0,NaN,NaN,1,1,0,0,1139,1050,0,0,NaN
1,8543168465,C1AG79687,0,1,620.140528,CLOSED_FULLY_PROCESSED,0,0,0,0,1141,0,0,1100,3534.0
2,7308726237,nan,0,0,NaN,NaN,0,1,0,1040,1138,1050,0,0,NaN
3,9070724843,nan,0,0,NaN,NaN,0,1,0,1040,1138,1050,1089,0,3534.0
4,6274074693,nan,0,0,NaN,NaN,0,1,0,1040,1141,1050,0,1100,3534.0


In [168]:
df_merged.shape

(71512, 15)

In [169]:
df_car_sales.car_id.value_counts()

# merchant_id is different which causes duplicate entries

7623794326    2
5402284800    2
7096519625    2
4063580476    2
7613645214    2
             ..
2805485321    1
9403741772    1
3636330388    1
9376021039    1
5517395488    1
Name: car_id, Length: 69551, dtype: int64

In [170]:
df_car_details.car_id.value_counts()

# fuel_type and navigation_system are causing duplicate entries

9907511030    3
1550589125    3
4204449840    3
7613645214    3
8978100755    3
             ..
9764384875    1
4740781080    1
7310242299    1
3269394355    1
5517395488    1
Name: car_id, Length: 69551, dtype: int64

In [171]:
df_car_claims.car_id.value_counts()

# fuel_type and navigation_system are causing duplicate entries

1469750417    4
5149303881    4
7453031564    3
6267771382    3
5741286159    3
             ..
2168498850    1
9157300274    1
6694478254    1
7565842323    1
5517395488    1
Name: car_id, Length: 69551, dtype: int64